## 연도별 업종대분류를 기준으로 top10기업을 구하기
- top10 선정 기준은 총자산순이익률
    - 총자산이익률 = 당기순이익 / 총자산

# 1. 필요한 라이브러리 불러오기

In [3]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

# 2. 데이터 불러오기

In [4]:
df = pd.read_csv('./custom_data/feature_engineering_idurstry_code.csv', encoding='cp949')

In [5]:
df.shape

(109142, 79)

In [6]:
df.columns

Index(['사업자등록번호', '결산년월', '유동자산', '매출채권', '비유동자산', '유형자산', '자산총계', '유동부채',
       '비유동부채', '부  채  총  계', '자본금', '이익잉여금(결손금）', '자본총계', '매출액', '판매비와관리비',
       '영업이익（손실）', '법인세비용차감전순손익', '법인세비용', '당기순이익(손실)', '기업순이익률(%)',
       '유보액/총자산(%)', '유보액/납입자본(%)', '매출액총이익률(%)', '매출액영업이익률(%)', '매출액순이익률(%)',
       '수지비율(%)', '경상수지비율', '영업비율(%)', '금융비용대매출액비율(%', '금융비용대부채비율(%)',
       '금융비용대총비용비율(%', '부채비율(%)', '차입금의존도(%)', '자기자본비율(%)', '순운전자본비율(%)',
       '유동부채비율(%)', '비유동부채비율(%)', '부채총계대 매출액(%)', '총자본회전율(회)', '재고자산회전율(회)',
       '매출채권회전율(회)', '매입채무회전율(회)', '미수금', '매출원가', '무형자산', '재고자산', '기업접두명',
       '기업명', '기업접미명', '기업영문명', '업종', '기업규모', '공기업구분 1.공기업 2.일반기업',
       '개인법인구분 1.법인 2.개인', '본점지점구분 1.본점 2.지점', '국외투자법인여부', '벤처기업여부', '상장코드',
       '산업코드차수', '산업코드1', '산업코드2', '산업코드3', '공공기관유형', '중견기업보호여부', '본점기업코드',
       '설립일자', '설립구분', '상장일자', '주요사업내용', '국가명', '홈페이지URL', '대표자명', '직원수',
       '종료일자', '시작일자', '휴폐업구분', '상태발생일자', '업종중분류', '업종대분류'],
      dtype='object')

# 3. 총자산이익률 계산하기

## 3.1. 계산 전 특이사항

In [7]:
df.loc[df['사업자등록번호'] == 4168194212][['결산년월', '자산총계', '당기순이익(손실)', '기업명', '휴폐업구분', '상태발생일자']]

,결산년월,자산총계,당기순이익(손실),기업명,휴폐업구분,상태발생일자
75303,20181231,15860687.0,-1948774.0,유앤에스에셋,폐업,20201031.0
75304,20191231,79172.0,6901133.0,유앤에스에셋,폐업,20201031.0


In [8]:
df.loc[df['사업자등록번호'] == 6498800195][['결산년월', '자산총계', '당기순이익(손실)', '기업명', '휴폐업구분', '상태발생일자']]

,결산년월,자산총계,당기순이익(손실),기업명,휴폐업구분,상태발생일자
102872,20181231,10130378.0,-539965.0,동양팰리스,폐업,20210205.0
102873,20191231,30011.0,995130.0,동양팰리스,폐업,20210205.0


- 폐업한 기업의 경우 폐업 전에 청산의 이유로 자산이 심하게 줄고 당기순이익이 급상승하는 패턴을 보이는 기업이 있다.
- 따라서 추후 총자산순이익율의 상위 10개를 구할 때는 폐업하지 않은 기업으로 범위를 좁힌다.

## 3.2. 계산을 위해 새로운 데이터프레임 만들기

In [9]:
# 필요한 피처만 가져오기
df_split = df[['사업자등록번호', '결산년월', '자산총계', '당기순이익(손실)', '기업명', '업종대분류', '휴폐업구분']]

In [10]:
# 정수형으로 되어있는 결산년월에서 연도 추출하기
def pop_year(value):
    return str(value)[:4]

df_split['년도'] = df_split['결산년월'].apply(pop_year)

In [11]:
# 자산총계와 당기순이익의 결측치는 0으로 보간
df_split.fillna(0, inplace=True)

In [12]:
# 공식에 따른 총자산순이익률 계산
df_split['총자산순이익률'] = df_split['당기순이익(손실)'] / df_split['자산총계']

In [13]:
df_split.head()

,사업자등록번호,결산년월,자산총계,당기순이익(손실),기업명,업종대분류,휴폐업구분,년도,총자산순이익률
0,1018100340,20181231,35014174.0,2509182.0,대일건설,부동산업,0,2018,0.071662
1,1018100340,20191231,36320470.0,2733421.0,대일건설,부동산업,0,2019,0.075258
2,1018100340,20201231,40601940.0,3406757.0,대일건설,부동산업,0,2020,0.083906
3,1018100340,20211231,38310842.0,3823473.0,대일건설,부동산업,0,2021,0.099801
4,1018100772,20181231,20792242.0,20159.0,쎈츄럴관광호텔,숙박 및 음식점업,0,2018,0.000970


In [14]:
df_split['업종대분류'].value_counts()

제조업                         44009
도매 및 소매업                    16734
부동산업                        13767
건설업                          9541
정보통신업                        4295
전문, 과학 및 기술 서비스업             4157
운수 및 창고업                     3876
사업시설 관리, 사업 지원 및 임대 서비스업     3696
금융 및 보험업                     2316
숙박 및 음식점업                    1937
예술, 스포츠 및 여가관련 서비스업          1681
수도, 하수 및 폐기물 처리, 원료 재생업      1158
협회 및 단체, 수리 및 기타 개인 서비스업      797
전기, 가스, 증기 및 공기 조절 공급업        580
교육 서비스업                       497
보건업 및 사회복지 서비스업                80
공공 행정, 국방 및 사회보장 행정            16
농업, 임업 및 어업                     5
Name: 업종대분류, dtype: int64

- 공공 행정, 국방 및 사회보장 행정과 농업, 임업 및 어업의 경우 개수가 너무 적다.
- top10을 뽑아낼 수도 없고, 적은 데이터의 평균을 구하는 것도 의미가 없다고 판단하여 둘의 평균총자산순이익률은 0으로 둔다.

In [15]:
# 평균총자산순이익률을 구할 때 공공 행정, 국방 및 사회보장 행정과 농업, 임업 및 어업 제외
indurstry_list = df_split['업종대분류'].unique().tolist()[:-2]

In [16]:
indurstry_list

['부동산업',
 '숙박 및 음식점업',
 '예술, 스포츠 및 여가관련 서비스업',
 '전문, 과학 및 기술 서비스업',
 '도매 및 소매업',
 '제조업',
 '정보통신업',
 '교육 서비스업',
 '운수 및 창고업',
 '금융 및 보험업',
 '건설업',
 '사업시설 관리, 사업 지원 및 임대 서비스업',
 '전기, 가스, 증기 및 공기 조절 공급업',
 '협회 및 단체, 수리 및 기타 개인 서비스업',
 '수도, 하수 및 폐기물 처리, 원료 재생업',
 '보건업 및 사회복지 서비스업']

In [17]:
# 자산총계가 0인 경우 계산하면 inf가 나오므로 0으로 변환
df_final = df_split.replace([np.inf, -np.inf], 0)

In [18]:
# 위에서 분석한 결과를 토대로 영업중인 기업에 대해서만 구함
df_final = df_final.loc[df_final['휴폐업구분'] == 0]

## 3.3. 연도별 업종대분류에 따른 총자산순이익률 top10 기업 구하고 평균 구하기

In [19]:
list_2018 = []
for indurstry in indurstry_list:    
    df_temp = df_final.loc[(df_final['년도'] == '2018') & (df_final['업종대분류'] == indurstry)].drop_duplicates(['기업명']).sort_values('총자산순이익률', ascending=False).head(10)
    mean_value = df_temp['총자산순이익률'].mean()
    list_2018.append(mean_value)
list_2018.append(0.0)
list_2018.append(0.0)

In [20]:
list_2019 = []
for indurstry in indurstry_list:
    df_temp = df_final.loc[(df_final['년도'] == '2019') & (df_final['업종대분류'] == indurstry)].drop_duplicates(['기업명']).sort_values('총자산순이익률', ascending=False).head(10)
    mean_value = df_temp['총자산순이익률'].mean()
    list_2019.append(mean_value)
list_2019.append(0.0)
list_2019.append(0.0)

In [21]:
list_2020 = []
for indurstry in indurstry_list:
    df_temp = df_final.loc[(df_final['년도'] == '2020') & (df_final['업종대분류'] == indurstry)].drop_duplicates(['기업명']).sort_values('총자산순이익률', ascending=False).head(10)
    mean_value = df_temp['총자산순이익률'].mean()
    list_2020.append(mean_value)
list_2020.append(0.0)
list_2020.append(0.0)

In [22]:
list_2021 = []
for indurstry in indurstry_list:
    df_temp = df_final.loc[(df_final['년도'] == '2021') & (df_final['업종대분류'] == indurstry)].drop_duplicates(['기업명']).sort_values('총자산순이익률', ascending=False).head(10)
    mean_value = df_temp['총자산순이익률'].mean()
    list_2021.append(mean_value)
list_2021.append(0.0)
list_2021.append(0.0)

# 4. 내보내기

In [23]:
# 아까 제외한 업종대분류 추가를 위해 다시 정의
concat_indurstry_list = df_split['업종대분류'].unique().tolist()

In [24]:
# 데이터프레임으로 만들기
concat_list = []
for mean_list, year in zip([list_2018, list_2019, list_2020, list_2021], ['2018', '2019', '2020', '2021']):
    df_temp = pd.DataFrame({'년도': [year]*18,
                            '업종대분류':concat_indurstry_list,
                            '평균총자산이익률':mean_list})
    concat_list.append(df_temp)
final = pd.concat(concat_list)

In [25]:
final

,년도,업종대분류,평균총자산이익률
0,2018,부동산업,0.885174
1,2018,숙박 및 음식점업,0.415017
2,2018,"예술, 스포츠 및 여가관련 서비스업",0.402173
3,2018,"전문, 과학 및 기술 서비스업",0.473636
4,2018,도매 및 소매업,0.998841
...,...,...,...
13,2021,"협회 및 단체, 수리 및 기타 개인 서비스업",0.235406
14,2021,"수도, 하수 및 폐기물 처리, 원료 재생업",0.768268
15,2021,보건업 및 사회복지 서비스업,0.032564
16,2021,"농업, 임업 및 어업",0.000000


In [26]:
final.to_csv('mean_top10_per_year.csv', encoding='cp949', index=False)